In [5]:
input_file_directory = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7//0 in_silico/Python/1)data_input/'
output_file_directory = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7/0 in_silico/Python/3)output/'

# I] Import

In [6]:
import pandas

# II] ATG7 isoform expression

In [9]:
%%time
# DL data ATG7 isoform expression
path = output_file_directory + "/ATG7/TCGA_GTEX/all_tissues/dataframes_for_input/df_ATG7_NormPrim_all.csv"
df_ATG7_NormPrim= pandas.read_csv(path, sep = ",")

#remove Survival information
df_ATG7_NormPrim.drop(df_ATG7_NormPrim.columns[[3, 4]], axis = 1, inplace = True)

print(df_ATG7_NormPrim.shape)
df_ATG7_NormPrim.head(2)

(16179, 20)
Wall time: 79.8 ms


,sample,Sample_Type,Primary_Site,ATG7,ATG7_1,ATG7_2,ATG7_3,ATG7_total,ATG7_1tpm,ATG7_2tpm,ATG7_3tpm,log2_p1_ATG7_total,log2_p1_ATG7_1,log2_p1_ATG7_2,ATG7_prot_tot,ATG7_1perc,ATG7_2perc,ATG7_3perc,ATG7_1norm,ATG7_2norm
0,GTEX-ZTTD-0326-SM-57WFW,Normal Tissue,Muscle,5.042,3.860,2.390,0.2642,32.944283,14.519306,5.240574,1.199970,5.085097,3.955992,2.641679,20.95985,69.271996,25.002916,5.725088,0.440723,0.159074
1,GTEX-PX3G-1626-SM-2S1PT,Normal Tissue,Muscle,4.252,2.322,1.956,1.1320,19.052710,4.999249,3.878848,2.190624,4.325725,2.584782,2.286540,11.06872,45.165557,35.043325,19.791118,0.262390,0.203585


In [10]:
#split to have Normal (GTEX) and Primary (TCGA)
df_ATG7_Normal = df_ATG7_NormPrim[df_ATG7_NormPrim['sample'].str.startswith('GTEX')]
print(len(df_ATG7_Normal))
df_ATG7_Primary = df_ATG7_NormPrim[df_ATG7_NormPrim['sample'].str.startswith('TCGA')]
print(len(df_ATG7_Primary))

7401
8778


## A) ATG7 isoforms + Whole gene expression (ensembl)

In [11]:
%%time
# DL data gene expression for Normal Tissue and Primary Tumors
path = output_file_directory + "dataframes_for_input/pancreas/"

ensembl_normal_pancreas_protcoding_expr= pandas.read_csv(path+ 'ensembl_normal_pancreas_protcoding_expr.tsv', sep = "\t")
print(ensembl_normal_pancreas_protcoding_expr.shape)

directory = "ensembl_primary_pancreas_protcoding_expr.tsv"
ensembl_primary_pancreas_protcoding_expr= pandas.read_csv(path+'ensembl_primary_pancreas_protcoding_expr.tsv' , sep = "\t")
print(ensembl_primary_pancreas_protcoding_expr.shape)
ensembl_primary_pancreas_protcoding_expr.head()

(167, 19383)
(178, 19383)
Wall time: 5.76 s


,sample,ENSG00000198888,ENSG00000198763,ENSG00000198804,ENSG00000198712,ENSG00000228253,ENSG00000198899,ENSG00000198938,ENSG00000198840,ENSG00000212907,...,ENSG00000160678,ENSG00000160679,ENSG00000143553,ENSG00000214193,ENSG00000196182,ENSG00000181817,ENSG00000116885,ENSG00000116898,ENSG00000119535,ENSG00000142694
0,TCGA-2J-AAB1-01,12.5979,12.7660,13.9326,13.5892,13.4221,13.6399,13.5917,13.4003,12.4744,...,5.2173,5.8480,4.8861,3.9127,4.1628,5.1575,2.9338,5.9086,2.3423,4.7549
1,TCGA-2J-AAB4-01,11.9508,12.2991,13.7032,13.9784,13.5229,13.5828,13.9554,12.9003,12.4813,...,3.6703,5.9189,4.9275,2.8381,4.5675,4.7793,2.7615,5.8590,2.7139,4.4054
2,TCGA-2J-AAB6-01,11.0837,11.8905,12.5723,12.9621,12.3837,12.5712,13.6005,11.8531,11.6337,...,1.9149,5.5756,5.1728,3.5174,4.1652,5.5085,2.2663,7.0333,2.4412,5.2377
3,TCGA-2J-AAB8-01,10.7231,12.7880,12.1531,13.7012,13.7530,13.4922,13.7765,13.2663,12.7046,...,3.2313,6.1002,5.4307,4.0136,4.0454,5.0246,3.4700,6.2627,3.2420,5.1048
4,TCGA-2J-AAB9-01,12.0252,11.9711,12.8020,12.7146,12.5884,12.6511,12.9294,12.8316,11.6517,...,1.9564,5.3345,4.8095,0.9935,4.0799,4.9127,2.6670,5.6153,3.3177,4.9397


In [12]:
#Preparation of the global dataframe with ATG7 isoform expression AND whole gene expression
data = [[ensembl_normal_pancreas_protcoding_expr, df_ATG7_Normal], [ensembl_primary_pancreas_protcoding_expr, df_ATG7_Primary]]

df_final = []

for df in data:
    #Merge the two df to have ATG7 isoform expression + whole gene expression 
  df_merged = pandas.merge(df[1], df[0], on= 'sample')

  #delete the sample column and select column
  df_merged = df_merged.drop("sample", 1)
  df_merged = df_merged.drop(df_merged.columns[5:19], axis=1)
  
  #save in list
  df_final.append(df_merged)

ensembl_normal_pancreas_protcoding_atg7_expr = df_final[0]
ensembl_primary_pancreas_protcoding_atg7_expr = df_final[1]

print(ensembl_normal_pancreas_protcoding_atg7_expr.shape)
print(ensembl_primary_pancreas_protcoding_atg7_expr.shape)
ensembl_normal_pancreas_protcoding_atg7_expr.head()

(167, 19387)
(178, 19387)


,Sample_Type,Primary_Site,ATG7,ATG7_1,ATG7_2,ENSG00000198888,ENSG00000198763,ENSG00000198804,ENSG00000198712,ENSG00000228253,...,ENSG00000160678,ENSG00000160679,ENSG00000143553,ENSG00000214193,ENSG00000196182,ENSG00000181817,ENSG00000116885,ENSG00000116898,ENSG00000119535,ENSG00000142694
0,Normal Tissue,Pancreas,3.259,2.064,0.6239,12.5552,12.7960,13.5489,13.4342,14.4641,...,2.0289,4.9924,4.0506,0.3907,4.6165,3.9719,2.1606,5.2028,4.8768,3.7204
1,Normal Tissue,Pancreas,3.216,2.635,-1.4700,13.0496,13.6030,13.7792,14.4138,15.0967,...,0.4016,5.7918,4.8610,1.4281,4.8949,5.0960,3.0498,5.6741,1.5661,5.4858
2,Normal Tissue,Pancreas,2.868,1.423,0.3346,13.1549,13.2598,13.4632,13.6357,14.4806,...,2.5263,4.9336,3.1637,-1.3548,4.2631,3.6646,1.3901,4.7814,1.1316,2.0076
3,Normal Tissue,Pancreas,2.830,2.095,-0.3940,12.6427,12.8969,13.5829,13.4833,13.7899,...,2.1444,5.4138,4.1986,-0.5543,4.3548,4.1194,1.8201,5.2563,4.6247,3.1212
4,Normal Tissue,Pancreas,2.822,1.683,0.2998,12.9135,12.6805,12.8769,12.9432,13.9998,...,0.8883,5.0202,3.8300,-1.2481,4.3709,4.3075,1.9822,5.1065,1.3454,2.9765


In [13]:
#save to csv
path = 'dataframes_for_input/pancreas/'
ensembl_normal_pancreas_protcoding_atg7_expr.to_csv(output_file_directory+path+'ensembl_normal_pancreas_atg7_proteinGE_all.tsv', sep = "\t", index=False)
ensembl_primary_pancreas_protcoding_atg7_expr.to_csv(output_file_directory+path+'ensembl_primary_pancreas_atg7_proteinGE_all.tsv', sep = "\t", index=False)

In [ ]:
##SYMBOL NOT UPDATED

## B) ATG7 isoforms + Whole gene expression (symbol)

In [ ]:
# %%time
# # DL data gene expression for Normal Tissue and Primary Tumors
# input_file_directory_gene_expr = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7/0 Python Analysis/gene_expr_preparation/'

# directory = input_file_directory_gene_expr + "symbol_normal_pancreas_protcoding_expr.tsv"
# symbol_normal_pancreas_protcoding_expr= pandas.read_csv(directory, sep = "\t")
# print(symbol_normal_pancreas_protcoding_expr.shape)

# directory = input_file_directory_gene_expr + "symbol_primary_pancreas_protcoding_expr.tsv"
# symbol_primary_pancreas_protcoding= pandas.read_csv(directory, sep = "\t")
# print(symbol_primary_pancreas_protcoding.shape)
# symbol_primary_pancreas_protcoding.head(2)

In [ ]:
# #Preparation of the global dataframe with ATG7 isoform expression AND whole gene expression
# data = [[symbol_normal_pancreas_protcoding, df_ATG7_Normal], [symbol_primary_pancreas_protcoding, df_ATG7_Primary]]

# df_final = []

# for df in data:
#     #Merge the two df to have ATG7 isoform expression + whole gene expression 
#   df_merged = pandas.merge(df[1], df[0], on= 'sample')

#   #delete the sample column and select column
#   df_merged = df_merged.drop("sample", 1)
#   df_merged = df_merged.drop(df_merged.columns[5:19], axis=1)
  
#   #save in list
#   df_final.append(df_merged)

# symbol_normal_pancreas_protcoding_atg7_expr = df_final[0]
# symbol_primary_pancreas_protcoding_atg7_expr = df_final[1]

# print(symbol_normal_pancreas_protcoding_atg7_expr.shape)
# print(symbol_primary_pancreas_protcoding_atg7_expr.shape)
# symbol_normal_pancreas_protcoding_atg7_expr.head()

In [ ]:
# #Why ATG7_x ?
# verif = symbol_normal_pancreas_protcoding_atg7_expr[symbol_normal_pancreas_protcoding_atg7_expr.columns[pandas.Series(symbol_normal_pancreas_protcoding_atg7_expr.columns).str.startswith('ATG7')]]
# print(verif.head(2))

In [ ]:
# data = [symbol_normal_pancreas_protcoding_atg7_expr, symbol_primary_pancreas_protcoding_atg7_expr]

# for df in data:
#     df.drop('ATG7_y', axis = 1, inplace=True) 
#     df.rename(columns = {'ATG7_x' : 'ATG7'}, inplace=True)
#     verif = df[df.columns[pandas.Series(df.columns).str.startswith('ATG7')]]
#     print(verif.head(2))

In [ ]:
#save to csv
# symbol_normal_pancreas_protcoding_atg7_expr.to_csv('symbol_normal_pancreas_atg7_ProteinGE_all.tsv', sep = "\t", index=False)
# symbol_primary_pancreas_protcoding_atg7_expr.to_csv('symbol_primary_pancreas_atg7_ProteinGE_all.tsv', sep = "\t", index=False)